# Shor's algorithm in a few lines of numpy

## Logical functions as linear transformations

Implementación de funciones lógicas como transformaciones matriciales entre los espacios completos de configuraciones de entrada y salida.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

plt.rcParams["figure.figsize"] = [3,3]

def showmat(m):
    plt.imshow(m); plt.axis('off');

In [ ]:
bit = [0,1]

In [ ]:
def decode(bits):
    r = np.zeros(2**len(bits),int)
    r[int(''.join([str(b) for b in bits ]),2)] = 1
    return r

def encode(oneshot):
    x = np.argmax(oneshot)
    l = np.round(np.log2(len(oneshot))).astype(int)
    fmt = f"{{x:0{l}b}}"
    return list([int(d) for d in fmt.format(x=x) ])

In [ ]:
import itertools
def bits(n):
    return itertools.product(*[bit for _ in range(n)])

def Oper(l):
    return np.array([decode(x) for x in l]).T

In [ ]:
def tp(A,B):
    return np.vstack([np.hstack(x) for x in np.tensordot(A,B,axes=0)])

def tps(As):
    if len(As) == 1:
        return As[0]
    else:
        return tp(As[0],tps(As[1:]))

Las puertas lógicas más comunes:

In [ ]:
WH = np.array([[1, 1],
               [1,-1]])/np.sqrt(2)

And = Oper([ [1 if x==1 and y==1 else 0] for x,y in bits(2) ])

Not  = Oper([ [1-x] for x, in bits(1) ])

Id = Not@Not

CNot = Oper([ [x,y if x==0 else 1-y] for x,y in bits(2) ])

Toffoli = Oper([ [x1, x2, 1-y if x1==1 and x2==1 else y] for x1,x2,y in bits(3) ])

## Classical gates

Verificamos que el orden de expansión es coherente.

In [ ]:
list(enumerate(bits(3)))

In [ ]:
list(map(decode,bits(3)))

In [ ]:
list(enumerate(map(encode, map(decode,bits(3)))))

In [ ]:
showmat(Oper([[x,y,z] for x,y,z in bits(3)]))

La función tps hace el tensor product y por tanto actúa como combinación en paralelo.

In [ ]:
for x,y,z in bits(3):
    print(x,y,z,encode(tps([Not,Id,Id])@decode([x,y,z])))

In [ ]:
Test  = Oper([ [x, y, z, z] for x,y,z in bits(3) ])

In [ ]:
showmat(Test)

In [ ]:
for x,y,z in bits(3):
    print(x,y,z,encode(Test@decode([x,y,z])))

Empezamos probando operaciones clásicas:

In [ ]:
plt.figure(figsize=(3,3))
showmat(And)

In [ ]:
for x,y in bits(2):
    print(x,y,encode(And@decode([x,y])))

In [ ]:
Or = Not @ And @ tps([Not,Not])

In [ ]:
plt.figure(figsize=(3,3))
showmat(Or)

In [ ]:
for x,y in bits(2):
    print(x,y,encode(Or@decode([x,y])))

La gracia está en combinar circuitos fijos, expandiendo entradas adecuadamente con tensor products.

## Adder

Construimos un sumador de 4 bits encadenando 4 de 1 bit:

In [ ]:
adder = Oper([( (x+y+s)%2,(x+y+s)//2) for s,x,y in bits(3) ])

In [ ]:
for s,x,y in bits(3):
    print(s,x,y,encode(adder@decode([s,x,y])))

Para conectarlos hay que dejar pasar las entradas y salidas en los canales adecuados en cada etapa:

In [ ]:
step1 = tps([adder,Id,Id,Id,Id,Id,Id])
showmat(step1); plt.show()
step2 = tps([Id,adder,Id,Id,Id,Id])
showmat(step2); plt.show()
step3 = tps([Id,Id,adder,Id,Id])
showmat(step3); plt.show()
step4 = tps([Id,Id,Id,adder])
showmat(step4); plt.show()

In [ ]:
adder4 = step4 @ step3 @ step2 @ step1
print(adder4.shape)
plt.figure(figsize=(8,3))
showmat(adder4)

Construimos la entrada alternando los bits de cada número, con los bits más significativos al final.

In [ ]:
def dec(x):
    return sum([v * 2**k for k,v in enumerate(reversed(x))])

In [ ]:
dec([1,1,0])

In [ ]:
def binary(num,length=4):
    fmt = '{:0'+str(length)+'b}'
    return  [int(c) for c in fmt.format(num)]

In [ ]:
binary(6,8)

In [ ]:
def rev(x): return list(reversed(x))

In [ ]:
a = 8
b = 7
ab = [0]+list(np.array(list(zip(reversed(binary(a)),reversed(binary(b))))).flatten())
ab

In [ ]:
encode(adder4 @ decode(ab))

In [ ]:
c = dec(rev(_))
c, c==a+b

Cada fila es un posible resultado, y los unos en ella indican los estados de entrada que lo producen. Cada columna solo tiene un uno.

In [ ]:
c = 7

pos = dec(rev(binary(c,5)))

In [ ]:
list(np.where(adder4[pos])[0])

In [ ]:
bs = binary(100,9)
print(bs)
bs[0], dec(list(reversed(bs[1::2]))), dec(list(reversed(bs[2::2])))

In [ ]:
bs = binary(280,9)
print(bs)
bs[0], dec(list(reversed(bs[1::2]))), dec(list(reversed(bs[2::2])))

## Uncertainty

Estas matrices de transformación son [matrices estocásticas](https://en.wikipedia.org/wiki/Stochastic_matrix), transforman densidades de probabilidad en densidades de probabilidad. Son probabilidades condicionadas, Cada columna suma 1.

Recordemos que producto matriz vector implementa la contracción P(y) = Sum P(y|x) P(x).

Los ejemplos anteriores son circuitos deterministas, por tanto las columnas no solo suman 1 sino que cada elemento de la base de estados de entrada produce sin ambiguedad una configuración de salida. Eso sí, es completamente normal que varios estados de entrada vayan al mismo de salida. Cada fila contiene las configuraciones que la activan.

Podemos analizar son eigensystem y svd estas matrices y se saca información interesante.

Con esta operación construimos un bit completamente incierto:

In [ ]:
erase = np.array([[1,1],
                  [1,1]])/2

In [ ]:
erase @ erase @ [0.2, 0.8]

En la operación anterior vamos a meter un bit incierto:

In [ ]:
probs = adder4 @ tps([Id,erase,Id,Id,Id,Id,Id,Id,Id]) @ decode(ab)
probs

In [ ]:
for ik,p in enumerate(probs):
    k = dec(rev(binary(ik,5)))
    if p >0:
        print(k,p)

O sea, (8 ó 9) + 7 = 15 ó 16

Con dos bits inciertos:

In [ ]:
probs = adder4 @ tps([Id,erase,Id,Id,Id,Id,erase,Id,Id]) @ decode(ab)
probs

In [ ]:
for ik,p in enumerate(probs):
    k = dec(rev(binary(ik,5)))
    if p >0:
        print(k,p)

O sea, (8 ó 9) + (3 ó 7) = 11 ó 12 ó 15 ó 16

In [ ]:
plt.figure(figsize=(8,3))
showmat(adder4 @ tps([Id,erase,Id,Id,Id,Id,erase,Id,Id]))

## Reversible computation

Si la matriz tiene inversa significa que la computación se puede deshacer, del estado final se puede volver al de partida. La matriz de suma del ejemplo anterior claramente no es invertible a menos que nos las arreglemos para mantener las entradas, explícita o implícitameante en el resultado.

Afortunadamente existen juegos universales de puertas lógicas reversibles, lo cual implica que en principio se puede computar sin consumir energía. La que se haya consumido se recupera deshaciendo la operación.

## Quantum gates

In [ ]:
plt.figure(figsize=(3,3))
showmat(CNot)

In [ ]:
for x,y in bits(2):
    print(x,y,encode(CNot@decode([x,y])))

In [ ]:
plt.figure(figsize=(3,3))
showmat(Toffoli)

In [ ]:
for c1,c2,y in bits(3):
    print(c1,c2,y,encode(Toffoli@decode([c1,c2,y])))

Reversible And

In [ ]:
for x,y in bits(2):
    print(x,y,encode(Toffoli@decode([x,y,0])))

Reversible Or

In [ ]:
ROr = tps([Not,Not,Not]) @ Toffoli @ tps([Not,Not,Id])

In [ ]:
for x,y in bits(2):
    print(x,y,encode(ROr@decode([x,y,0])))

## Deutchs-Jozsa

El ejemplo más simple de computación cuántica. Podemos determinar con una sola llamada si una función desconocida (tenemos su implementación oculta en una caja negra) que solo puede ser constante o "balanceada".

In [ ]:
# two WH gates in parallel for two bits
mix = tp(WH,WH)

def konst(x):
    return 1

def balanced(x):
    return 1 if x == 1 else 0

fun = balanced
#fun = konst

def xor(x,y):
    return 1 if x!=y else 0

# creates a reversible operation with an auxiliary input
reverK = Oper([( x, xor(y, konst(x)) ) for x,y in bits(2) ])

reverB = Oper([( x, xor(y, balanced(x)) ) for x,y in bits(2) ])

# check the operation and the order of bits
for x,y in bits(2):
    xs, yf = encode(reverK @ decode([x,y]))
    print (x,y, xs == x, yf == xor(y,konst(x)))
    xs, yf = encode(reverB @ decode([x,y]))
    print (x,y, xs == x, yf == xor(y,balanced(x)))

El primer bit de la salida nos da la solución: 0 = konst, 1 = balanced

In [ ]:
amps = mix @ reverK @ mix @ decode([0,1])
print('Amplitudes:', amps)

probs = np.abs(amps)**2

print('probabilities:')
for k,v in zip(bits(2), probs):
    if v >0:
        print(k,v)

In [ ]:
amps = mix @ reverB @ mix @ decode([0,1])
print('Amplitudes:', amps)

probs = np.abs(amps)**2

print('probabilities:')
for k,v in zip(bits(2), probs):
    if v >0:
        print(k,v)

In [ ]:
# with the identity in the auxiliary qbit it remains uncertain
amps = tp(WH,Id) @ reverB @ mix @ decode([0,1])
print('Amplitudes:', amps)

probs = np.abs(amps)**2

print('probabilities:')
for k,v in zip(bits(2), probs):
    if v >0:
        print(k,v)

In [ ]:
showmat(reverK)

In [ ]:
showmat(reverB)

## Shor

La factorización de enteros se reduce a encontrar una raíz cuadrada modular no trivial de la unidad, que a su vez se reduce a encontrar el período de una secuencia.

Vamos a construir el circuito para $f(x)=a^x \mod N$

(Empezamos con un registro n=4 para comprobar los cálculos.)

In [ ]:
a = 13
N = 15

n = 4
q = 4

def f(x):
    r = a**x % N
    return r, binary(r,q)

In [ ]:
for k in range(2**n):
    print(k, f(k))

Se observa la periodicidad que el algoritmo tendrá que detectar.

Construimos el circuito que la implementa, que produce la misma entrada y el resultado de la función. En una implementación física real esto habría que hacerlo con puertas lógicas reversibles. Es la parte más complicada.

In [ ]:
expmod =Oper([ xs + f(dec(xs))[1] for zs in bits(n+q) for xs in [list(zs[:n])]])

In [ ]:
showmat(expmod)

Verificamos que funciona correctamente con la organizacion de bits establecida.

In [ ]:
bs = encode(expmod @ decode(binary(14,n)+[0]*q))
print(bs)
dec(bs[:n]), dec(bs[n:])

In [ ]:
bs = encode(expmod @ decode(binary(11,n)+[0]*q))
print(bs)
dec(bs[:n]), dec(bs[n:])

Alimentamos el circuito con una superposición de todas las entradas:

In [ ]:
amps = expmod @ (tps([WH]*n + [Id]*q) @ decode([0]*(n+q)))
amps

Si observamos todos los bits del resultado, puede salir cualquier configuración de entrada con su salida asociada.

In [ ]:
def shprobs(amps,tol=1):
    probs = np.abs(amps)**2
    for k,v in zip(bits(n+q), probs):
        if v>tol/100:
            print(f'{dec(k[:n]):2} -> {dec(k[n:]):2}   {100*v:.2f}%')

In [ ]:
print('Probabilities:')
shprobs(amps)

Se obtienen exactamente las mismas probabilidades si se introduce un valor incierto clásico (usando el operadore `erase` anterior en vez de la puerta de Walsh-Hadamard). Esto significa que si desconocemos completamente qué entrada concreta se ha introducido, la salida puede ser cualquiera de las posibles con igual probabilidad.

En el caso cuántico se introduce un estado de superposición perfectamente definido y conocido, que se transforma, y al medirse en la base computacional se proyecta alguno de los resultados posibles.

In [ ]:
# Partial measurement of the bits in ks

def measure(state, ks):
    n = round(np.log2(len(state)))
    r = np.random.choice(np.arange(len(state)), p=np.abs(state)**2)
    print(r)
    xs = binary(r,n)
    print(xs)
    obs = np.array(xs)[ks]
    print(obs)
    newamps = np.array([ a if np.array_equal(np.array(bs)[ks] , obs) else 0 for bs, a in zip(bits(n), state) ])
    newamps = newamps/np.linalg.norm(newamps)
    return newamps

La primera idea clave del algoritmo de Shor es que al observar el valor de la función el estado de los qbits no observados, los que copian la entrada, queda en una superposición de los valores que producen este resultado concreto observado.

In [ ]:
collapsed = measure(amps, list(range(n,n+q)))

shprobs(collapsed)

En el caso clásico, esto nos diría que una de esas entradas es la que se introdujo concretamente en el circuito. En el caso cuántico tenemos un estado que mantiene todas las posibilidades. Si lo observamos obtendríamos una de ellas, igual que en el caso clásico.

Si de alguna manera pudiéramos medir estos qbits varias veces sin alterar el estado, obtendríamos diferentes valores con una sola ejecución de la exponenciación modular y podríamos deducir el período (la diferencia entre ellos es un múltiplo del período). Pero esto es físicamente imposible, no se puede clonar un estado cuántico. Habría que repetir el proceso ejecutando de nuevo la función desde el principio. En casos realistas de números grandes es muy improbable que se repita el resultado.

In [ ]:
collapsed = measure(amps, list(range(n,n+q)))

shprobs(collapsed)

La segunda clave del algoritmo de Shor es aplicar la transformada de Fourier a la parte del estado que contiene todas las entradas que producen el valor de salida observado, para determinar el período.

Hay que aumentar el número de qbits del registro que contiene la entrada para que se produzca un número suficiente de repeticiones. Se supone que debe ser $N^2 < 2^n < 2N^2$, pero en alguno de estos experimentos parece que funciona con valores menores.

In [ ]:
n = 6

expmod =Oper([ xs + f(dec(xs))[1] for zs in bits(n+q) for xs in [list(zs[:n])]])
amps = expmod @ (tps([WH]*n + [Id]*q) @ decode([0]*(n+q)))
print('Probabilities')
shprobs(amps)

In [ ]:
collapsed = measure(amps, list(range(n,n+q)))

Queda una superposición de los valores de entrada que producen el mismo resultado:

In [ ]:
plt.rcParams["figure.figsize"] = [8,3]
plt.plot(collapsed);

In [ ]:
pos = np.where(abs(collapsed)>0.1)[0]
print(pos)
print(pos[1:] - pos[:-1])
(pos[1]-pos[0])/2**q

(El período en el espacio expandido va multiplicado por el tamaño del otro registro.)

Como comprobación, extraemos las amplitudes de las configuraciones no observadas.

In [ ]:
def showprobs2():
    sa = np.zeros(2**n)
    print('Probabilities:')
    for k,a in zip(bits(n+q), collapsed):
        x = dec(k[:n])
        v = np.abs(a)**2
        sa[x] += a
        if v >0:
            print(f'{x:3} -> {dec(k[n:]):3}   {100*v:.2f}%')

    plt.bar(np.arange(len(sa)),np.abs(sa),width=0.5);
    plt.xlabel('x'); plt.ylabel('amp');
    return sa

In [ ]:
sa = showprobs2()

The Quantum Fourier Transform aplica la TF a la secuencia de amplitudes de un estado cuántico, ordenadas con la enumeración de binaria de los qubits... Se puede realizar físicamente con puertas de forma eficiente.

In [ ]:
def QFT(n):
    N = 2**n
    w = np.exp(1j*2*np.pi/N)
    r = np.array([[ w**(k*j) for k in range(N)] for j in range(N)]) / np.sqrt(N)
    return r

In [ ]:
abs(QFT(4)@np.conj(QFT(4).T) - np.eye(16)).max()

In [ ]:
showmat(np.real(QFT(5)))

In [ ]:
plt.figure(figsize=(6,3))
pf = np.abs(QFT(n) @ sa)**2
plt.plot(pf);
np.where(pf>1/100)

Since the period probably will seldom be an exact divisor of the length we need the convergents. We include here a simple implementation to compute the sequence of convergents of the continuous fraction expansion of a given fraction.

In [ ]:
def cf_expansion(n, d):
    e = []

    q = n // d
    r = n % d
    e.append(q)

    while r != 0:
        n, d = d, r
        q = n // d
        r = n % d
        e.append(q)

    return e


def convergents(e):
    n = [] # Nominators
    d = [] # Denominators

    for i in range(len(e)):
        if i == 0:
            ni = e[i]
            di = 1
        elif i == 1:
            ni = e[i]*e[i-1] + 1
            di = e[i]
        else: # i > 1
            ni = e[i]*n[i-1] + n[i-2]
            di = e[i]*d[i-1] + d[i-2]

        n.append(ni)
        d.append(di)
        yield (ni, di)

In [ ]:
print('Probabilities:')
for j,v in enumerate(pf):
    if v > 1/100:
        cs = list(convergents(cf_expansion(j,2**n)))
        print(f'{100*v:6.2f}%   {j:3}  {cs}')

With the candidates we verify that we have found the modular square root of one. 

In [ ]:
a**2 % N, a**4 % N

And finally we obtain the factors:

In [ ]:
from math import gcd

r = 4
p = gcd(a**(r//2)-1, N)

p, N//p, N%p

Lo que ocurre se ve casi mejor en el espacio completo. Preparamos el circuito para otra factorización:

In [ ]:
a = 19
N = 21

n = 6
q = 5

for k in range(2*N):
    print(k, f(k))
print('...')

expmod =Oper([ xs + f(dec(xs))[1] for zs in bits(n+q) for xs in [list(zs[:n])]])

amps0 = expmod @ (tps([WH]*n + [Id]*q) @ decode([0]*(n+q)))
print('Probabilities')
shprobs(amps0)

amps = tps([QFT(n)]+[Id]*q) @ (expmod @ (tps([WH]*n + [Id]*q) @ decode([0]*(n+q))))
print('\nWith QFT')
shprobs(amps,tol=1)

Repitiendo el experimento varias veces, aunque el valor de la función sea distinto, el resultado de la TF es siempre un múltiplo del período.

In [ ]:
probs0 = np.abs( measure(amps0, list(range(n,n+q))) )**2
plt.plot(probs0)
plt.show()
probs = np.abs( measure(amps, list(range(n,n+q))) )**2
plt.plot(probs);

In [ ]:
for k,v in zip(bits(n+q), probs):
    j = dec(k[:n])
    if v>5/100:
        cs = list(convergents(cf_expansion(j,2**n)))
        print(f'{100*v:6.2f}%   {j:3} - {dec(k[n:]):2}:  {cs}')
        for _,d in cs:
            if  a**d % N == 1:
                r = d
                break
print(r)
p = gcd(a**(r//2)-1, N)
p, N//p, N%p

FFT for non-integer frequencies:

In [ ]:
def shqft():
    x = np.zeros(256)
    x[5::9] = 1
    plt.plot(x);
    plt.title(f'period=9,  length=256,  true freq={256/9:.2f},  peaks={sum(x>0.5)}')
    plt.show()

    f = abs(np.fft.ifft(x))
    plt.plot(f,'.-');
    plt.title(f'FFT big peaks at {list(np.where(f>0.05)[0])}')
    plt.show()

In [ ]:
shqft()